# Baseline
1. 레이블을 제외한 컬럼의 결측치 최빈값으로 처리
2. 질적 변수 수치화
3. 레이블이 없는 데이터 제거

In [1]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [2]:
train = pd.read_parquet('./train.parquet').drop("ID", axis=1)
test = pd.read_parquet('./test.parquet').drop("ID", axis=1)
sample_submission = pd.read_csv('./dts/sample_submission.csv')

In [3]:
NaN_col = []

for i in range(len(train.columns)):
    if(sum(train[train.columns[i]].notna())<1000000):
        NaN_col.append(train.columns[i])
        
print(NaN_col)

for col in NaN_col:
    mode = train[col].mode()[0]
    train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(mode)

print('Done')

['Estimated_Departure_Time', 'Estimated_Arrival_Time', 'Origin_State', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Delay']
Done


In [4]:
qual_col = []

for i in range(len(train.dtypes)):
    if (train.dtypes[i] == 'object'):
        qual_col.append(train.columns[i])
        
print(qual_col)
qual_col.remove('Delay')
print(qual_col)

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
            
    test[i] = le.transform(test[i])
    
print('Done.')

['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number', 'Delay']
['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']
Done.


In [5]:
train = train.dropna()

column_number = {}
for i,column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

Done.


In [12]:
# train = train.drop('Delay', axis=1)
train.to_parquet('baseline_train.parquet', index=False)
test.to_parquet('baseline_test.parquet', index=False)

# Preprocessing_1

1. 레이블을 제외한 컬럼의 결측치 처리<br>
    Origin_State, Destination_State -> 다른 행의 데이터를 참조하여 설정
2. 질적 변수 수치화
3. 레이블 비지도학습으로 처리

In [84]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [271]:
train = pd.read_parquet('./train.parquet').drop('ID', axis=1)
test = pd.read_parquet('./train.parquet').drop('ID', axis=1)

In [268]:
train.head()

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,None
1,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,None
2,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,None
3,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,None
4,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,None


In [274]:
# 모든 값이 0 인 Cancelled, Diverted 컬럼 제거
train['Cancelled'].value_counts(),train['Diverted'].value_counts()
train.drop(['Cancelled','Diverted'], axis=1, inplace=True)
test.drop(['Cancelled','Diverted'], axis=1, inplace=True)

In [275]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 16 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Month                     1000000 non-null  int64  
 1   Day_of_Month              1000000 non-null  int64  
 2   Estimated_Departure_Time  890981 non-null   float64
 3   Estimated_Arrival_Time    890960 non-null   float64
 4   Origin_Airport            1000000 non-null  object 
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Origin_State              890985 non-null   object 
 7   Destination_Airport       1000000 non-null  object 
 8   Destination_Airport_ID    1000000 non-null  int64  
 9   Destination_State         890921 non-null   object 
 10  Distance                  1000000 non-null  float64
 11  Airline                   891080 non-null   object 
 12  Carrier_Code(IATA)        891010 non-null   object 
 13  Carrier_ID(DOT)           89

## Origin_State, Destination_State 전처리
- Airport or Airport_ID를 기반으로 결측치 처리

In [276]:
# Origin_State 처리 - Origin_Airport 혹은 Origin_Airport_ID 를 기반으로 Origin_State의 결측치 처리
origin = train[['Origin_Airport','Origin_Airport_ID','Origin_State']]
# Origin_Airport 사용
iata = list(origin['Origin_Airport'].unique())
state_dict = {}

for code in tqdm(iata):
    val = origin[origin['Origin_Airport'] == code]['Origin_State'].value_counts().index[0]
    key = code
    state_dict[key] = val

origin_ = pd.DataFrame(state_dict, index=[0]).transpose().reset_index()
origin_.columns=['Origin_Airport','Origin_State']

100%|██████████| 374/374 [00:14<00:00, 26.39it/s]


In [277]:
train = pd.merge(train, origin_, how='inner', left_on='Origin_Airport', right_on='Origin_Airport').drop('Origin_State_x', axis=1).rename(columns={'Origin_State_y':'Origin_State'})

In [278]:
# Destination_State 처리 - Destination_Airport 혹은 Destination_Airport_ID 를 기반으로 Destination_State의 결측치 처리
Destination = train[['Destination_Airport','Destination_Airport_ID','Destination_State']]
# Destination_Airport 사용
iata = list(Destination['Destination_Airport'].unique())
state_dict = {}

for code in tqdm(iata):
    key = code
    # Destination_Airport가 'YNG' 일 경우는 Destination_State 정보가 없어 직접 검색한 정보를 기반으로 처리
    if key == 'YNG':
        state_dict[key] = 'Ohio'
    else:
        val = Destination[Destination['Destination_Airport'] == code]['Destination_State'].value_counts().index[0]
        state_dict[key] = val

Destination_ = pd.DataFrame(state_dict, index=[0]).transpose().reset_index()
Destination_.columns=['Destination_Airport','Destination_State']

100%|██████████| 375/375 [00:14<00:00, 26.45it/s]


In [279]:
train = pd.merge(train, Destination_, how='inner', left_on='Destination_Airport', right_on='Destination_Airport').drop('Destination_State_x', axis=1).rename(columns={'Destination_State_y':'Destination_State'})

In [284]:
# 컬럼의 순서 원위치
train = train[pd.read_parquet('./train.parquet').drop(['ID','Cancelled','Diverted'], axis=1).columns]
train.head(3)

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,4,15,NaN,NaN,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,None
1,7,16,550.0,715.0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N733SA,None
2,1,13,1405.0,1535.0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N204WN,None


In [285]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 16 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Month                     1000000 non-null  int64  
 1   Day_of_Month              1000000 non-null  int64  
 2   Estimated_Departure_Time  890981 non-null   float64
 3   Estimated_Arrival_Time    890960 non-null   float64
 4   Origin_Airport            1000000 non-null  object 
 5   Origin_Airport_ID         1000000 non-null  int64  
 6   Origin_State              1000000 non-null  object 
 7   Destination_Airport       1000000 non-null  object 
 8   Destination_Airport_ID    1000000 non-null  int64  
 9   Destination_State         1000000 non-null  object 
 10  Distance                  1000000 non-null  float64
 11  Airline                   891080 non-null   object 
 12  Carrier_Code(IATA)        891010 non-null   object 
 13  Carrier_ID(DOT)           89

## Airline, Carrier_Code(IATA), Carrier_ID(DOT) 전처리
- Airline, Carrier_Code(IATA)의 결측치 값 : None
- Carrier_ID(DOT)의 결측치 값 : nan

In [410]:
# Carrier_ID(DOT)의 결측치 값 0으로 임시 처리

hmm = train[['Airline','Carrier_Code(IATA)','Carrier_ID(DOT)','Tail_Number']]
hmm['Carrier_ID(DOT)'].fillna(0, inplace=True)
hmm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Airline             891080 non-null   object 
 1   Carrier_Code(IATA)  891010 non-null   object 
 2   Carrier_ID(DOT)     1000000 non-null  float64
 3   Tail_Number         1000000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 38.1+ MB


C:\Users\user\AppData\Local\Temp\ipykernel_20412\4096829298.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hmm['Carrier_ID(DOT)'].fillna(0, inplace=True)


In [ ]:
# 각 Airline은 고유한 Carrier_ID(DOT)를 가지는 것을 확인
lst = list(hmm['Airline'].unique())
lst.remove(None)

i = 0
count = 0
ind = []

for airline in tqdm(lst):
    check = hmm[hmm['Airline'] == airline]
    check = list(check['Carrier_ID(DOT)'].unique())
    try:
        check.remove(0)
    except:
        print('Not None')
    print(check)
    if len(check) != 1:
        count += 1
        ind.append(i)

    i += 1

print(count)
print(ind)

In [384]:
# Carrier_ID(DOT) 처리 - Airline을 기반으로 Carrier_ID(DOT)의 결측치 처리
DOT = hmm[['Airline','Carrier_ID(DOT)']]

airlines = list(hmm['Airline'].unique())
airlines.remove(None)
state_dict = {}

for airline in tqdm(airlines):
    key = airline
    check = hmm[hmm['Airline'] == key]
    check = list(check['Carrier_ID(DOT)'].unique())
    try:
        check.remove(0)
    except:
        pass
    val = check[0]
    state_dict[key] = val

DOT_ = pd.DataFrame(state_dict, index=[0]).transpose().reset_index()
DOT_.columns=['Airline','Carrier_ID(DOT)']

100%|██████████| 28/28 [00:00<00:00, 28.75it/s]


In [411]:
hmm = pd.merge(hmm, DOT_, how='left', left_on='Airline', right_on='Airline')
hmm['Carrier_ID(DOT)_y'].fillna(0, inplace=True)

In [412]:
for i in tqdm(range(len(hmm))):
    if np.logical_and(hmm['Carrier_ID(DOT)_x'][i]==0,hmm['Carrier_ID(DOT)_y'][i]!=0):
        hmm['Carrier_ID(DOT)_x'][i] = hmm['Carrier_ID(DOT)_y'][i]

  0%|          | 0/1000000 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_20412\2495575261.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hmm['Carrier_ID(DOT)_x'][i] = hmm['Carrier_ID(DOT)_y'][i]
100%|██████████| 1000000/1000000 [00:17<00:00, 57220.53it/s]


In [413]:
hmm = hmm.rename(columns={'Carrier_ID(DOT)_x':'Carrier_ID(DOT)'}).drop('Carrier_ID(DOT)_y', axis=1)
hmm.head(2)

,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number
0,Southwest Airlines Co.,WN,19393.0,N7858A
1,Southwest Airlines Co.,WN,19393.0,N733SA


In [414]:
def zero_to_nan(x):
    if x == 0: return np.nan
    else: return x

hmm['Carrier_ID(DOT)'] = hmm['Carrier_ID(DOT)'].apply(zero_to_nan)
hmm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Airline             891080 non-null   object 
 1   Carrier_Code(IATA)  891010 non-null   object 
 2   Carrier_ID(DOT)     988117 non-null   float64
 3   Tail_Number         1000000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 38.1+ MB


In [387]:
hmm2[hmm2['Carrier_ID(DOT)_x'].isna()]
pd.merge(hmm, DOT_, how='left', left_on='Airline', right_on='Airline')[pd.merge(hmm, DOT_, how='left', left_on='Airline', right_on='Airline')['Carrier_ID(DOT)_x'] == 0]

,Airline,Carrier_Code(IATA),Carrier_ID(DOT)_x,Tail_Number,Carrier_ID(DOT)_y
9,Southwest Airlines Co.,WN,0.0,N7722B,19393.0
22,Southwest Airlines Co.,WN,0.0,N8318F,19393.0
28,None,None,0.0,N451WN,NaN
30,Southwest Airlines Co.,WN,0.0,N411WN,19393.0
31,Southwest Airlines Co.,WN,0.0,N412WN,19393.0
...,...,...,...,...,...
999941,None,UA,0.0,N862AS,NaN
999949,None,UA,0.0,N939SW,NaN
999952,SkyWest Airlines Inc.,UA,0.0,N925SW,20304.0
999967,SkyWest Airlines Inc.,UA,0.0,N435SW,20304.0


In [343]:
np.isnan(check[1])

True

In [322]:
len(hmm['Airline'].unique())

29

In [320]:
# hmm[hmm['Airline'] == hmm['Airline'].unique()[5]]['Carrier_Code(IATA)'].unique()
hmm[hmm['Airline'] == hmm['Airline'].unique()[9]]['Carrier_ID(DOT)'].unique()
# hmm[hmm['Carrier_Code(IATA)'] == 'UA']
# hmm[['Airline','Carrier_Code(IATA)']]

array([19977.,    nan])

In [260]:
iata = list(hmm['Tail_Number'].unique())
# iata.remove(None)
lst= []
state_dict = {}
count = 0
i = 0
for code in tqdm(iata):
    key = code
    val = hmm[hmm['Tail_Number'] == key]['Carrier_Code(IATA)'].unique()
    check = list(val)
    check.remove(None)
    if len(check) != 1:
        count += 1
        lst.append(i)
    i+=1
print(lst)
print(count)
    # if key == 'YNG':
    #     state_dict[key] = 'Ohio'
    # else:
    #     val = destination[destination['Destination_Airport'] == code]['Destination_State'].value_counts().index[0]
    #     state_dict[key] = val

 17%|█▋        | 1062/6430 [00:40<03:23, 26.44it/s]


ValueError: list.remove(x): x not in list

In [246]:
iata = list(hmm['Airline'].unique())
iata.remove(None)
# lst= []
state_dict = {}
i = 0
for code in tqdm(iata):
    if i not in lst:
        key = code
        val = hmm[hmm['Airline'] == key]['Carrier_Code(IATA)'].unique()
        val = list(val)
        val.remove(None)
        state_dict[key] = val[0]
    i+=1

state_dict

100%|██████████| 28/28 [00:00<00:00, 37.55it/s]


{'Southwest Airlines Co.': 'WN',
 'Delta Air Lines Inc.': 'DL',
 'JetBlue Airways': 'B6',
 'Envoy Air': 'AA',
 'United Air Lines Inc.': 'UA',
 'Frontier Airlines Inc.': 'F9',
 'American Airlines Inc.': 'AA',
 'Spirit Air Lines': 'NK',
 'Virgin America': 'VX',
 'Alaska Airlines Inc.': 'AS',
 'Air Wisconsin Airlines Corp': 'UA',
 'Allegiant Air': 'G4',
 'Endeavor Air Inc.': 'DL',
 'Comair Inc.': 'AA',
 'Commutair Aka Champlain Enterprises, Inc.': 'UA',
 'Horizon Air': 'AS',
 'Hawaiian Airlines Inc.': 'HA',
 'Capital Cargo International': 'AA',
 'Peninsula Airways Inc.': 'AS',
 'Empire Airlines Inc.': 'HA',
 'Cape Air': 'UA'}

In [247]:
temp_ = pd.DataFrame(state_dict, index=[0]).transpose().reset_index()
temp_.columns=['Airline','Carrier_Code(IATA)']
temp_

,Airline,Carrier_Code(IATA)
0,Southwest Airlines Co.,WN
1,Delta Air Lines Inc.,DL
2,JetBlue Airways,B6
3,Envoy Air,AA
4,United Air Lines Inc.,UA
5,Frontier Airlines Inc.,F9
6,American Airlines Inc.,AA
7,Spirit Air Lines,NK
8,Virgin America,VX
9,Alaska Airlines Inc.,AS


In [253]:
hmm2 = pd.merge(hmm, temp_, how='left', left_on='Airline',right_on='Airline')

In [252]:
hmm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Airline             891080 non-null   object 
 1   Carrier_Code(IATA)  891010 non-null   object 
 2   Carrier_ID(DOT)     891003 non-null   float64
 3   Tail_Number         1000000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 38.1+ MB


In [254]:
hmm2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Airline               891080 non-null   object 
 1   Carrier_Code(IATA)_x  891010 non-null   object 
 2   Carrier_ID(DOT)       891003 non-null   float64
 3   Tail_Number           1000000 non-null  object 
 4   Carrier_Code(IATA)_y  698015 non-null   object 
dtypes: float64(1), object(4)
memory usage: 45.8+ MB


In [258]:
hmm2[np.logical_and(hmm2['Carrier_Code(IATA)_x'].isna(), hmm2['Carrier_Code(IATA)_y'].notna())]

,Airline,Carrier_Code(IATA)_x,Carrier_ID(DOT),Tail_Number,Carrier_Code(IATA)_y
21,Southwest Airlines Co.,None,19393.0,N500WR,WN
27,Southwest Airlines Co.,None,19393.0,N725SW,WN
32,Southwest Airlines Co.,None,19393.0,N409WN,WN
36,Southwest Airlines Co.,None,19393.0,N8537Z,WN
48,Southwest Airlines Co.,None,NaN,N7718B,WN
...,...,...,...,...,...
999826,United Air Lines Inc.,None,19977.0,N36280,UA
999860,Cape Air,None,20253.0,N14834,UA
999875,Cape Air,None,NaN,N45838,UA
999882,United Air Lines Inc.,None,NaN,N33294,UA
